In [180]:
import xarray as xr
import pandas as pd
import panel as pn
from sklearn import datasets, linear_model
import numpy as np
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import Legend, LegendItem, Line, Circle

In [181]:
output_notebook()

Loading BokehJS ...

In [183]:
file_name_mod = "sodankyla_ifs-ecmwf_2018-02-01-2018-03-31_00_day2.nc"
file_name_obs = "sodankyla_obs_2018-02-01_to_2018-03-31.nc"

#opens the file
ds_mod = xr.open_dataset(file_name_mod)
ds_obs = xr.open_dataset(file_name_obs)

#select varibale to read
#default selects 2m temp, will be a drop down menu with more choices later 
variable = "tas"

var_obs = ds_obs[variable].to_dataframe()
var_mod = ds_mod[variable].to_dataframe()

#rename column names to easier differentiate between values when combined
var_obs = var_obs.rename(columns={variable:"var_obs"})
var_mod = var_mod.rename(columns={variable:"var_mod"})

In [185]:
#merges the dataframes based on matching time points, gets rid of all mismatches for time
merged = var_obs.merge(var_mod, left_index=True, right_index=True, how='inner')

In [186]:
#do linear regression and add that to plot
x = merged["var_obs"].values
y = merged["var_mod"].values

x = x.reshape(len(x),1)
y = y.reshape(len(y),1)

regr = linear_model.LinearRegression()
regr.fit(x,y)

LinearRegression()

In [191]:
p = figure(title="scatter 1: "+variable, x_axis_label = "observation", y_axis_label = "Model")
p.circle('var_obs','var_mod',source=merged,fill_alpha=0.2,size=10, color = 'blue')
p.line(x.reshape(len(x),),regr.predict(x).reshape(len(regr.predict(x)),),line_width=2, color = 'red')


GlyphRenderer(id='4488', ...)

In [192]:
show(p)